# Robintrack Bar Chart Race
You can create a bar chart race of the most popular stocks on Robinhood using data from [robintrack.net](https://robintrack.net/data-download). Simply download the data, we'll get it in wide format where the index is the date and each column is a stock symbol, then use the bar_chart_race package to create the graph.

In [1]:
# change directory to where you've downloaded the data
%cd /Users/josephbell/Downloads/tmp2/popularity_export

/Users/josephbell/Downloads/tmp2/popularity_export


In [65]:
# imports
import bar_chart_race as bcr
import datetime as dt
from datetime import datetime
import glob
import os
import pandas as pd
import pandas_datareader.data as web
import warnings

In [3]:
# generates a list of symbols in a python list from the csvs
stock_list = ['.'.join(filename.split('.')[:-1]) for filename in os.listdir("/Users/josephbell/Downloads/tmp2/popularity_export") if os.path.isfile(os.path.join('/Users/josephbell/Downloads/tmp2/popularity_export', filename))]
# sorts the list alphabetically
stock_list.sort()
# print(stock_list)
print('Number of stocks:', len(stock_list))

Number of stocks: 8460


In [4]:
%%time

# clean data
def cleanPopularity(indir='/Users/josephbell/Downloads/tmp2/popularity_export'):
    '''Cleans data by adding a symbol name column, separating the date out,
       and dropping the timestamp.'''
    os.chdir(indir)
    fileList = glob.glob('*')
    for filename in fileList:
        df = pd.read_csv(filename)
        # using csv file name and creating a new column 'Symbol'
        df['Symbol'] = os.path.splitext(os.path.basename(filename))[0]
        # separating date and timestamp
        new = df["timestamp"].str.split(" ", n=1, expand=True)
        df["Date"] = new[0]
        df["Time"] = new[1]
        df.drop(columns=['timestamp'], inplace=True)
        # dropping duplicate dates
        df.drop_duplicates(subset='Date', keep='first', inplace=True)
        df.drop(columns=['Time'], inplace=True)
        df.to_csv('cleaned' + filename, index=False,encoding='utf-8-sig')
        
cleanPopularity()

CPU times: user 7min 4s, sys: 1min 43s, total: 8min 48s
Wall time: 8min 59s


In [5]:
# check if the data has been properly cleaned
df = pd.read_csv('cleanedAAPL.csv')
df.head()

,users_holding,Symbol,Date
0,150785,AAPL,2018-05-02
1,145510,AAPL,2018-05-03
2,143747,AAPL,2018-05-04
3,141424,AAPL,2018-05-05
4,141436,AAPL,2018-05-06


In [18]:
%%time
# joins all of the csv files into one dataframe
# set working directory
os.chdir('/Users/josephbell/Downloads/tmp2/popularity_export') # change to your path
# find all csv files in the folder
# use glob pattern matching -> extension = 'csv'
# save result in list -> all_filenames
extension = 'csv'
all_filenames = [i for i in glob.glob('*.{}'.format(extension))]
# combine all files in the list
popularity_data = pd.concat([pd.read_csv(f) for f in all_filenames], sort=True)

CPU times: user 1min 58s, sys: 1min 6s, total: 3min 4s
Wall time: 6min 27s


In [19]:
print(popularity_data.shape)
popularity_data.head()

(133138528, 4)


,Date,Symbol,timestamp,users_holding
0,2018-05-02,OAS,NaN,1524.0
1,2018-05-03,OAS,NaN,1517.0
2,2018-05-04,OAS,NaN,1522.0
3,2018-05-05,OAS,NaN,1520.0
4,2018-05-06,OAS,NaN,1520.0


In [20]:
# dropping timestamp column
popularity_data.drop('timestamp', axis=1, inplace=True)
print(popularity_data.shape)
popularity_data.head()

(133138528, 3)


,Date,Symbol,users_holding
0,2018-05-02,OAS,1524.0
1,2018-05-03,OAS,1517.0
2,2018-05-04,OAS,1522.0
3,2018-05-05,OAS,1520.0
4,2018-05-06,OAS,1520.0


In [10]:
# drop null values
popularity_data = popularity_data.dropna()
print(popularity_data.shape)
popularity_data.head()

(5821323, 3)


,Date,Symbol,users_holding
0,2018-05-02,OAS,1524.0
1,2018-05-03,OAS,1517.0
2,2018-05-04,OAS,1522.0
3,2018-05-05,OAS,1520.0
4,2018-05-06,OAS,1520.0


# Get Names to Match Symbols
We have the symbols for each stock, but that can become hard to follow especially when a stock you don't know the symbol for shows up on the list. You can download the name that goes with each symbol [here](https://robintrack.net/leaderboard) and click on download [full leaderboard as a csv.](https://robintrack.net/api/most_popular.csv?limit=100000)

In [21]:
names = pd.read_csv('/Users/josephbell/Downloads/leaderboard_2020-06-17_06-00-00.csv')
names = names.rename(columns = {'symbol': 'Symbol'})
names.drop('popularity', axis=1, inplace=True)
print(names.shape)
names.head()

(8170, 2)


,Symbol,name
0,F,Ford Motor
1,GE,GE
2,AAL,American Airlines
3,DIS,Disney
4,DAL,Delta Air Lines


In [22]:
# merging symbols with names
names_bc = popularity_data.merge(names, how ='inner', on = ('Symbol'))
print(names_bc.shape)
names_bc.head()

(5294479, 4)


,Date,Symbol,users_holding,name
0,2018-05-02,OAS,1524.0,Oasis Petroleum
1,2018-05-03,OAS,1517.0,Oasis Petroleum
2,2018-05-04,OAS,1522.0,Oasis Petroleum
3,2018-05-05,OAS,1520.0,Oasis Petroleum
4,2018-05-06,OAS,1520.0,Oasis Petroleum


In [23]:
# drop null values
names_bc = names_bc.dropna()
print(names_bc.shape)
names_bc.head()

(5059655, 4)


,Date,Symbol,users_holding,name
0,2018-05-02,OAS,1524.0,Oasis Petroleum
1,2018-05-03,OAS,1517.0,Oasis Petroleum
2,2018-05-04,OAS,1522.0,Oasis Petroleum
3,2018-05-05,OAS,1520.0,Oasis Petroleum
4,2018-05-06,OAS,1520.0,Oasis Petroleum


# Dealing with Carnival
For some reason when I pivot the data to convert it to wide format, the Carnival Cruise Lines (CCL) data gets corrupted so I save it to a csv file. We'll need this later.

In [24]:
carnival = names_bc[names_bc.values == 'CCL']
print(carnival.shape)
carnival.tail()

(763, 4)


,Date,Symbol,users_holding,name
5221970,2020-06-15,CCL,499581.0,Carnival
5221971,2020-06-16,CCL,502004.0,Carnival
5221972,2020-06-17,CCL,502422.0,Carnival
5221973,2020-06-18,CCL,502941.0,Carnival
5221974,2020-06-19,CCL,501509.0,Carnival


In [25]:
# convert to a csv
carnival.to_csv('carnival.csv')

In [26]:
names_bc['Date'].nunique()

763

In [31]:
# pivot data so that the index is the date and each column is a company name
names_pivot = pd.pivot_table(names_bc, index=['Date'], columns=['name'])
# fill null values with 0 users holding
names_pivot = names_pivot.fillna(0)
names_pivot

users_holding                                                  \
name       1-800-Flowers 10x Genomics    111 1347 PIH 180 Degree Capital   
Date                                                                       
2018-05-02         747.0          0.0    0.0     33.0              770.0   
2018-05-03         756.0          0.0    0.0     33.0              762.0   
2018-05-04         758.0          0.0    0.0     33.0              760.0   
2018-05-05         755.0          0.0    0.0     33.0              753.0   
2018-05-06         755.0          0.0    0.0     33.0              753.0   
...                  ...          ...    ...      ...                ...   
2020-06-15        2106.0       1500.0  393.0     19.0              994.0   
2020-06-16        2097.0       1524.0  400.0     19.0             1021.0   
2020-06-17        2075.0       1542.0  396.0     19.0             1020.0   
2020-06-18        2070.0       1553.0  398.0     31.0             1027.0   
2020-06-19        2089.0       1558.0  396.0     26.0             1036.0   

                                                                             \
name       1Life Healthcare 1st Constitution Bank 1st Source 21Vianet Group   
Date                                                                          
2018-05-02              0.0                  24.0       21.0          114.0   
2018-05-03              0.0                  24.0       20.0          115.0   
2018-05-04              0.0                  24.0       20.0          116.0   
2018-05-05              0.0                  24.0       20.0          118.0   
2018-05-06              0.0                  24.0       20.0          118.0   
...                     ...                   ...        ...            ...   
2020-06-15           4323.0                  44.0       73.0          343.0   
2020-06-16           4312.0                  45.0       72.0          337.0   
2020-06-17           4306.0                  44.0       71.0          326.0   
2020-06-18           4305.0                  44.0       71.0          508.0   
2020-06-19           4295.0                  44.0       73.0          460.0   

                               ...                                         \
name       22nd Century Group  ...  iStar inTEST j2 Global nVent Electric   
Date                           ...                                          
2018-05-02            34789.0  ...   52.0  316.0      82.0            0.0   
2018-05-03            34723.0  ...   57.0  318.0      84.0            0.0   
2018-05-04            34729.0  ...   58.0  322.0      83.0            4.0   
2018-05-05            34477.0  ...   56.0  317.0      86.0           17.0   
2018-05-06            34477.0  ...   56.0  317.0      86.0           17.0   
...                       ...  ...    ...    ...       ...            ...   
2020-06-15            50758.0  ...  408.0  493.0     251.0          221.0   
2020-06-16            50898.0  ...  409.0  492.0     258.0          222.0   
2020-06-17            50830.0  ...  408.0  493.0     259.0          222.0   
2020-06-18            50821.0  ...  412.0  496.0     261.0          225.0   
2020-06-19            50857.0  ...  403.0  495.0     266.0          225.0   

                                                                        \
name       nnovator S&P 500 Power Buffer nnovator S&P 500 Ultra Buffer   
Date                                                                     
2018-05-02                           0.0                           0.0   
2018-05-03                           0.0                           0.0   
2018-05-04                           0.0                           0.0   
2018-05-05                           0.0                           0.0   
2018-05-06                           0.0                           0.0   
...                                  ...                           ...   
2020-06-15                           2.0                           2.0   
2020-06-16          

In [32]:
# save pivot table as a csv
names_pivot.to_csv('names_pivot2.csv')

# Use Excel
Edit the pivot table csv in excel by deleting the top row with users_holding. Delete cell A1 where name is, shift cells up and delete row 2 from cell B2 to the right and shift cells up. Save the file and we'll reload it in. Search the spreadsheet for 'carnvial'. You may notice that the numbers in this column do not match the numbers from the dataframe. You can insert the correct numbers from the carnival csv prior to pivoting. 

It's a mystery as to why the pivot caused this, but I found the error as Carivnal wasn't showing up on the most popular stocks when it should have. There may be other errors, but everything else looked normal from spot checks.  In some cases a symbol gets changed like in the case of Draft Kings where it briefly makes an appearance on the list in 2018 before it was public because another company had the symbol DKNG in 2018.

In [50]:
# reload the csv after editing it
df = pd.read_csv('/Users/josephbell/Downloads/tmp2/popularity_export/names_pivot2.csv')
print(df.shape)
df.head()

(763, 7646)


,Date,1-800-Flowers,10x Genomics,111,1347 PIH,180 Degree Capital,1Life Healthcare,1st Constitution Bank,1st Source,21Vianet Group,...,iStar,inTEST,j2 Global,nVent Electric,nnovator S&P 500 Power Buffer,nnovator S&P 500 Ultra Buffer,scPharmaceuticals,uniQure,vTv Therapeutics,|Xtrackers J.P. Morgan ESG USD High Yield Corporate Bond
0,5/2/18,747,0,0,33,770,0,24,21,114,...,52,316,82,0,0,0,117,0,1716,0
1,5/3/18,756,0,0,33,762,0,24,20,115,...,57,318,84,0,0,0,121,0,1664,0
2,5/4/18,758,0,0,33,760,0,24,20,116,...,58,322,83,4,0,0,118,0,1632,0
3,5/5/18,755,0,0,33,753,0,24,20,118,...,56,317,86,17,0,0,116,0,1617,0
4,5/6/18,755,0,0,33,753,0,24,20,118,...,56,317,86,17,0,0,116,0,1617,0


In [51]:
# make sure you're in the right directory
%cd /Users/josephbell/Downloads/tmp2/popularity_export

/Users/josephbell/Downloads/tmp2/popularity_export


# Get Trading Days
In order to reduce the time the bar chart race takes to complete and make it flow smoothly we don't need to look at every single calendar day. Instead we can just take trading days, which excludes weekends and holidays. We'll get the trading days from AAPL and then merge them with the popularity data, which is collected everyday.

In [52]:
# get trading dates by getting prices for one stock
def get_stock_data(): 
    tickers = ['AAPL']

    start = dt.datetime(2018, 5, 2)
    end = dt.datetime(2020, 6, 19)

    if not os.path.exists('pricedata'):
        os.makedirs('pricedata')
  
    for ticker in tickers:
        print(ticker)
        try : 
            df = web.DataReader(ticker, 'yahoo', start, end).drop(['High','Low','Open','Adj Close'], axis=1)
            df['Symbol'] = ticker
            print(df.head())
            df.to_csv('pricedata/{}.csv'. format(ticker))
            print(ticker, 'downloaded')
        except Exception as e: 
            pass
            # print(e, 'error')

print(get_stock_data())

AAPL
                 Close      Volume Symbol
Date                                     
2018-05-02  176.570007  66539400.0   AAPL
2018-05-03  176.889999  34068200.0   AAPL
2018-05-04  183.830002  56201300.0   AAPL
2018-05-07  185.160004  42451400.0   AAPL
2018-05-08  186.050003  28402800.0   AAPL
AAPL downloaded
None


In [53]:
# get trading dates from a single stock
dates = pd.read_csv('/Users/josephbell/Downloads/tmp2/popularity_export/pricedata/AAPL.csv')
dates.drop(['Close', 'Volume', 'Symbol'], axis=1, inplace=True)
dates['Date'] = pd.to_datetime(dates['Date'])
dates.head()

,Date
0,2018-05-02
1,2018-05-03
2,2018-05-04
3,2018-05-07
4,2018-05-08


In [56]:
# get date in matching format
df['Date'] = pd.to_datetime(df['Date'])
df.head()

,Date,1-800-Flowers,10x Genomics,111,1347 PIH,180 Degree Capital,1Life Healthcare,1st Constitution Bank,1st Source,21Vianet Group,...,iStar,inTEST,j2 Global,nVent Electric,nnovator S&P 500 Power Buffer,nnovator S&P 500 Ultra Buffer,scPharmaceuticals,uniQure,vTv Therapeutics,|Xtrackers J.P. Morgan ESG USD High Yield Corporate Bond
0,2018-05-02,747,0,0,33,770,0,24,21,114,...,52,316,82,0,0,0,117,0,1716,0
1,2018-05-03,756,0,0,33,762,0,24,20,115,...,57,318,84,0,0,0,121,0,1664,0
2,2018-05-04,758,0,0,33,760,0,24,20,116,...,58,322,83,4,0,0,118,0,1632,0
3,2018-05-05,755,0,0,33,753,0,24,20,118,...,56,317,86,17,0,0,116,0,1617,0
4,2018-05-06,755,0,0,33,753,0,24,20,118,...,56,317,86,17,0,0,116,0,1617,0


In [57]:
# spot check if carnival numbers are correct
df['Carnival']

0        1003
1        1001
2         997
3         998
4         998
        ...  
758    499581
759    502004
760    502422
761    502941
762    501509
Name: Carnival, Length: 763, dtype: int64

In [70]:
# merge on date to get rid of non-trading days
df2 = df.merge(dates, how='inner', on=('Date'))
print(df2.shape)
df2.head()

(526, 7646)


,Date,1-800-Flowers,10x Genomics,111,1347 PIH,180 Degree Capital,1Life Healthcare,1st Constitution Bank,1st Source,21Vianet Group,...,iStar,inTEST,j2 Global,nVent Electric,nnovator S&P 500 Power Buffer,nnovator S&P 500 Ultra Buffer,scPharmaceuticals,uniQure,vTv Therapeutics,|Xtrackers J.P. Morgan ESG USD High Yield Corporate Bond
0,2018-05-02,747,0,0,33,770,0,24,21,114,...,52,316,82,0,0,0,117,0,1716,0
1,2018-05-03,756,0,0,33,762,0,24,20,115,...,57,318,84,0,0,0,121,0,1664,0
2,2018-05-04,758,0,0,33,760,0,24,20,116,...,58,322,83,4,0,0,118,0,1632,0
3,2018-05-07,755,0,0,33,753,0,24,20,118,...,56,317,86,17,0,0,116,0,1617,0
4,2018-05-08,752,0,0,33,750,0,25,21,119,...,61,305,94,28,0,0,109,0,1545,0


In [71]:
# set date column as index
df2 = df2.set_index('Date')
df2.head()

,1-800-Flowers,10x Genomics,111,1347 PIH,180 Degree Capital,1Life Healthcare,1st Constitution Bank,1st Source,21Vianet Group,22nd Century Group,...,iStar,inTEST,j2 Global,nVent Electric,nnovator S&P 500 Power Buffer,nnovator S&P 500 Ultra Buffer,scPharmaceuticals,uniQure,vTv Therapeutics,|Xtrackers J.P. Morgan ESG USD High Yield Corporate Bond
Date,,,,,,,,,,,,,,,,,,,,,
2018-05-02,747,0,0,33,770,0,24,21,114,34789,...,52,316,82,0,0,0,117,0,1716,0
2018-05-03,756,0,0,33,762,0,24,20,115,34723,...,57,318,84,0,0,0,121,0,1664,0
2018-05-04,758,0,0,33,760,0,24,20,116,34729,...,58,322,83,4,0,0,118,0,1632,0
2018-05-07,755,0,0,33,753,0,24,20,118,34477,...,56,317,86,17,0,0,116,0,1617,0
2018-05-08,752,0,0,33,750,0,25,21,119,34427,...,61,305,94,28,0,0,109,0,1545,0


# Create the Bar Chart Race
Now that our data is in the correct format we can create our bar chart race! You can edit the parameters to customize your graph. A list of parameters can be found [here](https://www.dexplo.org/bar_chart_race/). Note: this does take a while to run. Once it is done you can scroll to the bottom of the notebook to view your chart or save it as a file under filename. 

In [72]:
warnings.simplefilter('ignore')
bcr.bar_chart_race(df=df2, n_bars=30, filename='Robinhood_barchart_race3.mp4', title='Most Popular Stocks on Robinhood by Users Holding',
                   period_fmt='%B %d, %Y', filter_column_colors=True,
                   period_summary_func=lambda v, r: {'x': .95, 'y': .09,
                   's': '@joebellowns',
                   'ha': 'right', 'size': 8, 'family': 'Helvetica'})